In [1]:
# !pip install import_ipynb --quiet
# !pip install anvil-uplink --quiet
# !pip install yfinance --quiet
# !pip install pandas_ta --quiet
# !pip install ipynb --quiet
# !pip install rpyc --quiet
# !pip install stable-baselines3 --quiet
# !pip install aspectlib

In [2]:
# !git clone https://github.com/gmshroff/algostrats.git

In [3]:
# %cd algostrats

Uncomment below if on Colab and using datasets from Kaggle

In [4]:
# from google.colab import files
# uploaded=files.upload()

In [5]:
# !mkdir /root/.kaggle
# !mv ./kaggle.json /root/.kaggle/.
# !chmod 600 /root/.kaggle/kaggle.json

In [6]:
# %mkdir data
# %cd data
# !kaggle datasets download -d gmshroff/marketdatafivemin
# %cd ../algostrats

In [4]:
colab=False
script=False
if not colab: DATAPATH='~/DataLocal/algo_fin_new/five_min_data/'
elif colab: DATAPATH='../algodata/'

Need to import algorithms from stable-baselines3

In [5]:
from stable_baselines3 import PPO,A2C,DQN
from stable_baselines3.common.vec_env import StackedObservations
from stable_baselines3.common.monitor import Monitor as Mon

/Users/rishabh/miniforge3/envs/metarl/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import warnings
warnings.simplefilter("ignore")

In [7]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle
from threading import Thread
import threading
from IPython import display
import time,getopt,sys,os

In [8]:
from feeds import BackFeed,DataFeed
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from featfuncs import add_global_indices_feed

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from synfeed.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from featfuncs.ipynb


In [9]:
from feed_env import Episode
import aspectlib

importing Jupyter notebook from feed_env.ipynb


### Trading strategies as agents

RL(++)StratAgents imported from ruleagents<br>

In [10]:
from rlagents import RLStratAgentDyn

importing Jupyter notebook from aiagentbase.ipynb


### Strategy Testing for RLStratAgent using BackTest

In [11]:
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate

importing Jupyter notebook from backtest.ipynb
importing Jupyter notebook from validation.ipynb


<b>Configuration for training RL model<b>

In [12]:
algorithm=PPO
synthetic=True #use synthetic data
simple='sinewave'
nw,nd=20,5 #for BackFeed

In [13]:
if script:
    try:
        opts,args = getopt.getopt(sys.argv[1:],"hl:f:d:m:s:w:t:p:u:",["load=","feed=","datafile=","modelname=","synthetic","weeks","training_steps","deploy","use_alt_data"])
    except getopt.GetoptError:
        print('rlagents_train.py -l <load:True/False> -f <scan:back/data> -d <datafile> -m <modelname> -s <synthetic> -w <weeks> -t <training_steps> -p <deploy>')
        sys.exit(2)
    load,feed,modelname=False,'back',''
    training_steps=50000 # if less then n_steps then n_steps is used
    deploy=True
    date=datetime.today().strftime('%d-%b-%Y')
    use_alt_data=False
    for opt, arg in opts:
        if opt == "-h":
            print('rlagents_train.py -l <load:True/False> -f <scan:back/data> -d <datafile> -m <modelname> -s <synthetic> -w <weeks> -t <training_steps> -p <deploy> -u <use_alt_data>')
            sys.exit()
        elif opt in ("-l", "--load"):
            load = (lambda x: True if x=='True' else False)(arg)
        elif opt in ("-f", "--feed"):
            feed = (lambda x: 'data' if x=='data' else 'back')(arg)
        elif opt in ("-d", "--datafile"):
            datafile = arg.split('/')[-1]
        elif opt in ("-m", "--modelname"):
            modelname = arg
        elif opt in ("-s", "--synthetic"):
            synthetic = (lambda x: True if x=='True' else False)(arg)
        elif opt in ("-w", "--weeks"):
            nw = int(arg)
        elif opt in ("-t", "--training_steps"):
            training_steps=int(arg)
        elif opt in ("-p", "--deploy"):
            deploy = (lambda x: True if x=='True' else False)(arg)
        elif opt in ("-u", "--use_alt_data"):
            use_alt_data = (lambda x: True if x=='True' else False)(arg)
    if len(opts)==0: 
        print('rlagents_train.py -l <load:True/False> -f <scan:back/data> -d <datafile> -m <modelname> -s <synthetic> -w <weeks> -t <training_steps> -p <deploy> -u <use_alt_data>')
        sys.exit()
    print(f"load:{load},feed:{feed},datafile:{datafile},modelname:{modelname},synthetic:{synthetic},weeks:{nw},training_steps:{training_steps},deploy:{deploy},use_alt_data:{use_alt_data}")
    loadfeed=load
    if feed=='data': datafeed=True
    else: datafeed=False

In [14]:
if not script:
    loadfeed=False
    datafeed=False
    datafile='augdata_16-Dec-2022_5m.csv'
    # modelname='RLA0.pth' # replace with modelname if model to be loaded
    modelname='hp_5.pth'
    date=datetime.today().strftime('%d-%b-%Y')
    deploy=True

In [15]:
n_steps=2048 # reduce for debugging only else 2048

In [16]:
def stringify(x):
    return pd.to_datetime(x['Datetime']).strftime('%d-%b-%Y')

In [17]:
import pickle
if not loadfeed and not datafeed:
    data=pd.read_csv('./capvol100.csv')
    tickers=list(data.iloc[0:50]['ticker'].values)
    print('Creating feed')
    feed=BackFeed(tickers=tickers,nd=nd,nw=nw,interval='5m',synthetic=synthetic)
    print('Processing feed')
    # add_addl_features_feed(feed,tickers=feed.tickers)
    # add_sym_feature_feed(feed,tickers=feed.tickers)
    add_global_indices_feed(feed)
    if colab: 
        with open('/tmp/btfeed.pickle','wb') as f: pickle.dump(feed,f)
    else: 
        with open('../algodata/btfeed.pickle','wb') as f: pickle.dump(feed,f)
elif loadfeed and not datafeed:
    if colab: 
        with open('/tmp/btfeed.pickle','rb') as f: feed=pickle.load(f)
    else: 
        with open('../algodata/btfeed_hp_5.pkl','rb') as f: feed=pickle.load(f)
        print('Loaded')

Creating feed
Processing feed
^NYA: No data found for this date range, symbol may be delisted
^IXIC: No data found for this date range, symbol may be delisted
^NYA: No data found for this date range, symbol may be delisted
LSEG.L: No data found for this date range, symbol may be delisted
^IXIC: No data found for this date range, symbol may be delisted
LSEG.L: No data found for this date range, symbol may be delisted
^NYA: No data found for this date range, symbol may be delisted
LSEG.L: No data found for this date range, symbol may be delisted
^IXIC: No data found for this date range, symbol may be delisted
^NYA: No data found for this date range, symbol may be delisted
^IXIC: No data found for this date range, symbol may be delisted
^NYA: No data found for this date range, symbol may be delisted
^IXIC: No data found for this date range, symbol may be delisted


In [18]:
if not loadfeed and datafeed:
    #DATAFILE=DATAPATH+'augdata_'+date+'_5m.csv'
    DATAFILE=DATAPATH+datafile
    print('Reading datafile')
    df=pd.read_csv(DATAFILE)
    if 'Date' not in df.columns: 
        print('Adding Date')
        df['Date']=df.apply(stringify,axis=1)
    print('Creating feed')
    feed=DataFeed(tickers=list(df.ticker.unique()[0:10]),dfgiven=True,df=df)
    print('Processing feed')
    add_addl_features_feed(feed,tickers=feed.tickers)
    add_sym_feature_feed(feed,tickers=feed.tickers)
    add_global_indices_feed(feed)
    if not colab: 
        with open('../../temp_data/btdatafeed.pickle','wb') as f: pickle.dump(feed,f)
    elif colab: 
        with open('/tmp/btdatafeed.pickle','wb') as f: pickle.dump(feed,f)
elif loadfeed and datafeed:
    if not colab: 
        with open('../../temp_data/btdatafeed.pickle','rb') as f: feed=pickle.load(f)
    elif colab:
        with open('/tmp/btdatafeed.pickle','rb') as f: feed=pickle.load(f)

In [19]:
def get_alt_data_live():
    aD={'gdata':feed.gdata}
    return aD

In [20]:
feed.data['SYN']

,datetime,Date,ticker,Open,High,Low,Close,Volume
2022-09-29 09:15:00,2022-09-29 09:15:00,29-Sep-2022,SYN,1.000000,1.000000,1.001742,1.001742,1.0
2022-09-29 09:20:00,2022-09-29 09:20:00,29-Sep-2022,SYN,1.002175,1.002175,1.003892,1.003892,1.0
2022-09-29 09:25:00,2022-09-29 09:25:00,29-Sep-2022,SYN,1.004316,1.004316,1.005981,1.005981,1.0
2022-09-29 09:30:00,2022-09-29 09:30:00,29-Sep-2022,SYN,1.006388,1.006388,1.007976,1.007976,1.0
2022-09-29 09:35:00,2022-09-29 09:35:00,29-Sep-2022,SYN,1.008360,1.008360,1.009844,1.009844,1.0
...,...,...,...,...,...,...,...,...
2023-02-23 15:05:00,2023-02-23 15:05:00,23-Feb-2023,SYN,1.061158,1.061158,1.061483,1.061483,1.0
2023-02-23 15:10:00,2023-02-23 15:10:00,23-Feb-2023,SYN,1.061564,1.061564,1.061889,1.061889,1.0
2023-02-23 15:15:00,2023-02-23 15:15:00,23-Feb-2023,SYN,1.061971,1.061971,1.062296,1.062296,1.0
2023-02-23 15:20:00,2023-02-23 15:20:00,23-Feb-2023,SYN,1.062377,1.062377,1.062702,1.062702,1.0


In [21]:
use_alt_data=False
agent=RLStratAgentDyn(algorithm,monclass=Mon,soclass=StackedObservations,verbose=1,win=5,
                   metarl=True,myargs=(n_steps,use_alt_data))
agent.use_memory=True #depends on whether RL algorithm uses memory for state computation
agent.debug=False
if use_alt_data: agent.set_alt_data(alt_data_func=get_alt_data_live)

Using cpu device
Wrapping the env in a DummyVecEnv.


In [22]:
agent.training=False

In [23]:
if modelname and os.path.exists('./saved_models/'+modelname): 
    agent.load_model(filepath='./saved_models/'+modelname)
    print(f'Loaded model from {modelname}')

Loaded model from hp_5.pth


In [24]:
@aspectlib.Aspect
def my_decorator(*args, **kwargs):
    # print("Got called with args: %s kwargs: %s" % (args, kwargs))
    # result = yield
    # print(" ... and the result is: %s" % (result,))
    state,rew,done,exit_type = yield
    # args[0].policy.logL+=[(state.keys(),rew,done,exit_type)]
    args[0].policy.reward((rew,done,{'exit_type':exit_type}))
    return state,rew,done,exit_type

In [25]:
aspectlib.weave(Episode, my_decorator, methods='env_step')

In [26]:
bt=Backtest(feed,tickers=feed.tickers,add_features=False,target=5,stop=5,txcost=0.001,
            loc_exit=True,scan=False,topk=5,deploy=deploy,save_dfs=False,
            save_func=None)

AttributeError: 'BackFeed' object has no attribute 'ndata'

In [51]:
agent.data_cols=agent.data_cols+['Date']

In [52]:
bt.run_all(tickers=feed.tickers,model=agent,verbose=False)

In [53]:
sum([bt.results[t][d]['tot'] for t in bt.results for d in bt.results[t]])

1161.820245109653

## Trading Plots

In [35]:
import plotly.graph_objects as go

In [36]:
import plotly.express as px

In [37]:
def annotate_action(rew,act,df):
    if rew[1]>=0:color='Green'
    else: color='Red'
    if act[0]==1:text='Buy'
    elif act[0]==-1:text='Sell'
    ann=dict(font=dict(color=color,size=15),x=df.index[rew[0]],y=df.iloc[rew[0]]['Close'],
             showarrow=True,text=text)
    return ann

In [38]:
def annotate_exit(rew,act,anns,df):
    if rew[1]>=0:color='Green'
    else: color='Red'
    X=[a['x'] for a in anns if a is not None]
    if df.index[rew[2]] in X: 
        idx=X.index(df.index[rew[2]])
        anns[idx]['text']='Ex&'+anns[idx]['text']
    else:
        anns+=[dict(font=dict(color=color,size=15),x=df.index[rew[2]],y=df.iloc[rew[2]]['Close'],
                    showarrow=True,text='Exit')]

In [39]:
def plot_ticker_date(bt,ticker,date):
    global fig
    df=feed.ndata[ticker][date]
    df=df.loc[df['Date']==date]
    fig = go.Figure(data=
        [go.Candlestick(x = df.index,
                        open  = df["Open"],
                        high  = df["High"],
                        low   = df["Low"],
                        close = df["Close"])]
    )
    reward=np.round(bt.results[ticker][date]["tot"],2)
    fig.update_layout(
        title=f'{ticker} on {date} return {reward}',
        yaxis_title="Price"
    )
    anns=[]
    for r,a in zip(bt.results[ticker][date]['rew'],bt.results[ticker][date]['acts']):
        anns+=[annotate_action(r,a,df)]
    for r,a in zip(bt.results[ticker][date]['rew'],bt.results[ticker][date]['acts']):
        anns+=[annotate_exit(r,a,anns,df)]
    for a in anns: 
        if a is not None: fig.add_annotation(a)
    fig.show()
    return fig

In [40]:
def combine_plotly_figs_to_html(plotly_figs, html_fname, include_plotlyjs='cdn', 
                                separator=None, auto_open=False):
    with open(html_fname, 'w') as f:
        f.write(plotly_figs[0].to_html(include_plotlyjs=include_plotlyjs))
        for fig in plotly_figs[1:]:
            if separator:
                f.write(separator)
            f.write(fig.to_html(full_html=False, include_plotlyjs=False))

    if auto_open:
        import pathlib, webbrowser
        uri = pathlib.Path(html_fname).absolute().as_uri()
        webbrowser.open(uri)

In [41]:
figs=[]
for t in bt.results:
    for d in bt.results[t]:
        figs+=[plot_ticker_date(bt,t,d)]

In [35]:
combine_plotly_figs_to_html(figs,'/tmp/charts.html',auto_open=True)